# <span style="color:#0F19C9">Contenido</span>

- [Hacer Web Scrapping](#hacer-web-scrapping)

# <span style="color:#0F19C9">Hacer Web Scraping</span>

Debido a que no encontramos una herramienta que pudiera descargar directamente la información de la API de TikTok, desarrollaremos nuestro propio script de Web Scrapping para extraer la información pública de los perfiles.

In [34]:
# Importar librerías para el manejo del dataframe
import time
import json
import pandas as pd
import datetime
import pytz
import re

# Importar librerías para web scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [330]:
# Configurar Selenium
driver = webdriver.Chrome()

In [331]:
# Función para hacer scroll hasta el final de la página
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_number = 0
    while scroll_number < 4:
        driver.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        scroll_number += 1
        if new_height == last_height:
            break
        last_height = new_height

In [332]:
# Función para extraer los primeros 20 videos de un perfil usando BeautifulSoup
def extract_profile_data(profile_url):
    driver.get(profile_url)
    time.sleep(5)

    # Hacer scroll para cargar todos los videos (20 en este caso)
    scroll_to_bottom(driver)

    # Usar BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extraer número de seguidores y likes
    followers = soup.find('strong', {'data-e2e': 'followers-count'}).text
    total_likes = soup.find('strong', {'data-e2e': 'likes-count'}).text

    # Extraer URLs de los videos
    video_elements = soup.find_all(
        'a', {'class': 'css-1g95xhm-AVideoContainer'}, limit=20)
    video_urls = [video['href']
                  for video in video_elements if ('/video/' in video['href']) and (profile_url in video['href'])]

    return followers, total_likes, video_urls[:20]

In [333]:
# Función para extraer la información de cada video usando BeautifulSoup
def extract_video_data(video_url):
    driver.get(video_url)
    driver.implicitly_wait(5)

    # Usar BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extraer el contenido del script con el id __UNIVERSAL_DATA_FOR_REHYDRATION__
    script_tag = soup.find(
        'script', {'id': '__UNIVERSAL_DATA_FOR_REHYDRATION__'})

    # Convertir el contenido del script a un objeto JSON
    data = json.loads(script_tag.string)

    # Navegar por el JSON para extraer la información del video
    try:
        video_info = data['__DEFAULT_SCOPE__']['webapp.video-detail']['itemInfo']['itemStruct']

        # Extraer los datos necesarios
        timestamp = video_info.get('createTime', None)
        description = video_info.get('desc', None)
        views = video_info.get('stats', {}).get('playCount', 0)
        likes = video_info.get('stats', {}).get('diggCount', 0)
        comments = video_info.get('stats', {}).get('commentCount', 0)
        shares = video_info.get('stats', {}).get('shareCount', 0)
        duration = video_info.get('video', {}).get('duration', None)
        try:
            content_type = 'video' if video_info.get('video') else 'image'
        except:
            content_type = 'image'

    except KeyError as e:
        print(f"Error al extraer datos del video: {e}")
        return None

    return {
        "timestamp": timestamp,
        "description": description,
        "views": views,
        "likes": likes,
        "comments": comments,
        "shares": shares,
        "duration": duration,
        "content_type": content_type
    }

In [334]:
# Función principal para extraer de múltiples perfiles
def scrape_tiktok_profiles(profiles):
    data = []

    for profile in profiles:
        followers, total_likes, video_urls = extract_profile_data(profile)

        for video_url in video_urls:
            video_data = extract_video_data(video_url)
            if video_data:
                video_data["profile"] = profile
                video_data["followers"] = followers
                video_data["total_likes"] = total_likes
                data.append(video_data)
            else:
                print('Foto sin estadísticas')

    return pd.DataFrame(data)

In [335]:
# Perfiles de TikTok a analizar
profiles = [
    "https://www.tiktok.com/@hermanosarizashowoficial",
    "https://www.tiktok.com/@urielhenaooficial",
    "https://www.tiktok.com/@tugrupodominio",
    "https://www.tiktok.com/@los5delnorte"
]

# Perfiles de referentes
# profiles = [
#     "https://www.tiktok.com/@yeison_jimenez1",
#     "https://www.tiktok.com/@pipebueno97",
#     "https://www.tiktok.com/@jessiuribeoficial",
#     "https://www.tiktok.com/@luisalfonso",
#     "https://www.tiktok.com/@lostigresdelnorte",
#     "https://www.tiktok.com/@bobbypulido425",
#     "https://www.tiktok.com/@grupo_intocable",
#     "https://www.tiktok.com/@lostucanesdetijuana"
# ]

# Ejecuta la función
df = scrape_tiktok_profiles(profiles)

# Cerrar driver
driver.quit()

# <span style="color:#0F19C9">Arreglar formatos</span>

Como la información se descargó directamente de los metadatos de las publicaciones será importante modificar sus formatos para realizar un análisis exitoso.

In [92]:
# Función para convertir timestamp a fecha y hora en Bogotá
def convert_timestamp_to_bogota(timestamp):
    bogota_tz = pytz.timezone('America/Bogota')
    dt = datetime.datetime.fromtimestamp(int(timestamp), tz=pytz.utc)
    return dt.astimezone(bogota_tz).strftime('%Y-%m-%d %H:%M:%S')

In [93]:
# Función para convertir valores con K y M a enteros
def convert_k_m_to_int(value):
    if 'K' in value:
        return int(float(value.replace('K', '')) * 1000)
    elif 'M' in value:
        return int(float(value.replace('M', '')) * 1000000)
    else:
        return int(value)

In [95]:
# Convertir marcas de tiempo
df['timestamp'] = df['timestamp'].apply(convert_timestamp_to_bogota)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Cambia los nombres de los perfiles
df['profile'] = df['profile'].replace({
    'https://www.tiktok.com/@hermanosarizashowoficial': 'Los Hermanos Ariza Show',
    'https://www.tiktok.com/@urielhenaooficial': 'Uriel Henao',
    'https://www.tiktok.com/@tugrupodominio': 'Grupo Dominio',
    'https://www.tiktok.com/@los5delnorte': 'Los 5 del Norte'
})

# Convierte los seguidores y total_likes a enteros
df['followers'] = df['followers'].apply(convert_k_m_to_int).astype(int)
df['total_likes'] = df['total_likes'].apply(convert_k_m_to_int).astype(int)

In [96]:
def count_words(text):
    '''Función para contar las palabras del caption'''
    return len(text.split())


def get_hashtags(text):
    '''Función para extraer los hashtags del caption'''
    return re.findall(r'#\w+', text)

In [97]:
# Aplicar funciones sobre los captions
df['description'] = df['description'].fillna('')
df['Palabras en Caption'] = df['description'].apply(count_words)
df['Hashtags'] = df['description'].apply(get_hashtags)
df['Número de hashtags'] = df['Hashtags'].apply(len)

In [99]:
# Organizar columnas
df = df[['profile', 'followers', 'total_likes', 'timestamp',
         'duration', 'views', 'likes', 'comments', 'shares',
         'description', 'Palabras en Caption', 'Hashtags',
         'Número de hashtags']]

# Renombrar columnas
df.columns = ['Artista', 'Seguidores', 'Likes totales', 'Fecha y Hora',
              'Duración', 'Visualizaciones', 'Likes', 'Comentarios',
              'Compartido', 'Caption', 'Palabras en Caption', 'Hashtags',
              'Número de hashtags']

In [101]:
# Guardar dataframe
df.to_csv('../Data/TikTokNorteño.csv', index=False)

# Mostrar 3 registro aleatorios
df.head(3)

,Artista,Seguidores,Likes totales,Fecha y Hora,Duración,Visualizaciones,Likes,Comentarios,Compartido,Caption,Palabras en Caption,Hashtags,Número de hashtags
0,Yeison Jiménez,1600000,13500000,2024-08-08 20:03:24,29,5500000,109000,313,1179,"A mí solo me usan pa arreglar relaciones 😅, me...",32,"[#nuevamusica, #lallamada, #yeisonjimenez, #vi...",5
1,Yeison Jiménez,1600000,13500000,2024-08-16 18:58:43,73,685800,29300,232,452,"Así se ve componer un palazo, ya le hizo La Ll...",20,"[#nuevamusica, #lallamada, #lanzamiento, #yeis...",5
2,Yeison Jiménez,1600000,13500000,2024-08-12 20:40:52,80,4800000,180200,754,4525,"Palazo como este, NINGUNO! 🥷🏻 Quién listo pa L...",17,"[#lallamada, #nuevamusica, #lanzamiento, #long...",6
